# **Experimental**: Run and get results

## Get results from an anonymous simulation : `anon_runandget`

Sometimes you want to run a simulation on an idf and get a particular result. There is no single function in `eppy` which can do that. In this experimental section we are exploring functions that will achieve this objectives. 

So what does this functionality look like:

(Let do some housekeeping first to run this notebook)

In [1]:
# the lines in this block are needed to run the code in this notebook
# you don't need them if you have eppy installed
import sys
# pathnameto_eppy = 'c:/eppy'
pathnameto_witheppy = '../'
sys.path.append(pathnameto_witheppy) 

Open the `idf` file (Your path names may be different on your machine)

In [2]:
fname = "/Applications/EnergyPlus-9-3-0/ExampleFiles/1ZoneEvapCooler.idf"
wfile = "/Applications/EnergyPlus-9-3-0/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"

import eppy
import witheppy.runandget as runandget
import pprint
pp = pprint.PrettyPrinter()

idf = eppy.openidf(fname, epw=wfile) # this is an easy way to open the idf
                                     # if you have trouble here, go back to 
                                     # the tutorial and see the longer manual 
                                     # way to open the file


Lets say you want to run a simulation and get just the `*.end` file. In reality we rarely get that file. But it is a small file, so it is easy to demonstrate using that file. 

The `getdict` dictionary defines what you want to extract from the results. Right now we want to extract the entire `*.end` file.

In [3]:
getdict = dict(
    end_file=dict(whichfile="end", entirefile=True),
)

# run and get the result. anon_runandget() will run the 
# simulation in a temprary file that will be deleted afer you get the results
fullresult = runandget.anon_runandget(idf, getdict)
pp.pprint(fullresult)    


/Applications/EnergyPlus-9-3-0/energyplus --weather /Applications/EnergyPlus-9-3-0/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw --output-directory /var/folders/sm/202yyhk50_s9p3s_4g2kqxvm0000gn/T/tmpjdaqrgog --idd /Applications/EnergyPlus-9-3-0/Energy+.idd --readvars /Users/santoshphilip/Documents/coolshadow/github/witheppy/docs/in.idf

{'end_file': {'entirefile': True,
              'result': 'EnergyPlus Completed Successfully-- 4 Warning; 0 '
                        'Severe Errors; Elapsed Time=00hr 00min  4.40sec\n',
              'whichfile': 'end'}}


We want only the results. So

In [4]:
print(fullresult['end_file']['result'])

EnergyPlus Completed Successfully-- 4 Warning; 0 Severe Errors; Elapsed Time=00hr 00min  4.40sec



This is great. But what if I want only part of a result file. Lets say I want I want the first table from the `html table file`. And I want it in a list format with rows and columns. Aha! Let us try this

In [5]:
getdict = dict(
    HTML_file=dict(whichfile="htm", tableindex=0, table=True),
)
# run and get the result. anon_runandget() will run the 
# simulation in a temprary file that will be deleted afer you get the results
fullresult = runandget.anon_runandget(idf, getdict)
pp.pprint(fullresult) 


/Applications/EnergyPlus-9-3-0/energyplus --weather /Applications/EnergyPlus-9-3-0/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw --output-directory /var/folders/sm/202yyhk50_s9p3s_4g2kqxvm0000gn/T/tmpu42rhn6f --idd /Applications/EnergyPlus-9-3-0/Energy+.idd --readvars /Users/santoshphilip/Documents/coolshadow/github/witheppy/docs/in.idf

{'HTML_file': {'entirefile': None,
               'result': ['Site and Source Energy',
                          [['',
                            'Total Energy [GJ]',
                            'Energy Per Total Building Area [MJ/m2]',
                            'Energy Per Conditioned Building Area [MJ/m2]'],
                           ['Total Site Energy', 18.06, 77.76, 77.76],
                           ['Net Site Energy', 18.06, 77.76, 77.76],
                           ['Total Source Energy', 57.2, 246.26, 246.26],
                           ['Net Source Energy', 57.2, 246.26, 246.26]]],
               'table': True,
               

Sweet !! Let us print just the result.

In [6]:
pp.pprint(fullresult['HTML_file']['result'])

['Site and Source Energy',
 [['',
   'Total Energy [GJ]',
   'Energy Per Total Building Area [MJ/m2]',
   'Energy Per Conditioned Building Area [MJ/m2]'],
  ['Total Site Energy', 18.06, 77.76, 77.76],
  ['Net Site Energy', 18.06, 77.76, 77.76],
  ['Total Source Energy', 57.2, 246.26, 246.26],
  ['Net Source Energy', 57.2, 246.26, 246.26]]]


Ha ! this is fun. What if I want last column, but just the last two values, but lets use the table name instead of the table index

In [7]:
getdict = dict(
    twocells=dict(
        whichfile="htm",
        # tableindex=0,  # or tablename
        tablename="Site and Source Energy",  # tableindex takes priority if both given
        cells=[[-2, -1], [-2, -1]],  # will return 2 cells
    )
)
# run and get the result. anon_runandget() will run the 
# simulation in a temprary file that will be deleted afer you get the results
fullresult = runandget.anon_runandget(idf, getdict)
pp.pprint(fullresult)


/Applications/EnergyPlus-9-3-0/energyplus --weather /Applications/EnergyPlus-9-3-0/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw --output-directory /var/folders/sm/202yyhk50_s9p3s_4g2kqxvm0000gn/T/tmpoqvq1f83 --idd /Applications/EnergyPlus-9-3-0/Energy+.idd --readvars /Users/santoshphilip/Documents/coolshadow/github/witheppy/docs/in.idf

{'twocells': {'cells': [[-2, -1], [-2, -1]],
              'entirefile': None,
              'result': ['Site and Source Energy', [246.26, 246.26]],
              'tablename': 'Site and Source Energy',
              'whichfile': 'htm'}}


What if I want the contents of `*.end` **AND** the first table from the html table file. Here you go.

In [8]:
getdict = dict(
    HTML_file=dict(whichfile="htm", tableindex=0, table=True),
    end_file=dict(whichfile="end", entirefile=True),
)
fullresult = runandget.anon_runandget(idf, getdict)
pp.pprint(fullresult)


/Applications/EnergyPlus-9-3-0/energyplus --weather /Applications/EnergyPlus-9-3-0/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw --output-directory /var/folders/sm/202yyhk50_s9p3s_4g2kqxvm0000gn/T/tmp4wrmt7w4 --idd /Applications/EnergyPlus-9-3-0/Energy+.idd --readvars /Users/santoshphilip/Documents/coolshadow/github/witheppy/docs/in.idf

{'HTML_file': {'entirefile': None,
               'result': ['Site and Source Energy',
                          [['',
                            'Total Energy [GJ]',
                            'Energy Per Total Building Area [MJ/m2]',
                            'Energy Per Conditioned Building Area [MJ/m2]'],
                           ['Total Site Energy', 18.06, 77.76, 77.76],
                           ['Net Site Energy', 18.06, 77.76, 77.76],
                           ['Total Source Energy', 57.2, 246.26, 246.26],
                           ['Net Source Energy', 57.2, 246.26, 246.26]]],
               'table': True,
               

## What can `getdict` get ?

Let us look at some examples. For instance you can any file generated by the simulation. So far we got the `end` file and the `htm` file. What do we call them

They are known by these names.

In [9]:
pp.pprint(runandget.resulttypes)

['audit',
 'bnd',
 'dxf',
 'eio',
 'end',
 'err',
 'eso',
 'mdd',
 'mtd',
 'mtr',
 'rdd',
 'shd',
 'htm',
 'tab',
 'sqlerr',
 'csv',
 'mcsv',
 'expidf',
 'sql',
 'rvaudit']


Most of them are the actual file extendions. There are two `csv` files. so we get them by `csv` and`mcsv`, where `mcsv` will get you the `meter.csv` file and `csv` will get you the regular csv file. Some examples of getting the entire file.

Whenever possible `entirefile=True` will return a text file. In the case of `htm` and `sql`, the file will be read in binary mode.

In [10]:
getdict = dict(end_file=dict(whichfile="end", entirefile=True))
getdict = dict(HTML_file=dict(whichfile="htm", entirefile=True))
getdict = dict(csv_file=dict(whichfile="csv", entirefile=True))
getdict = dict(eio_file=dict(whichfile="eio", entirefile=True))
getdict = dict(sql_file=dict(whichfile="sql", entirefile=True)) 

OK ! That gets you the whole file. 

What are the varaitions in getting a partial data from a file. Below is a list of all of them. Let's start with the html file

In [11]:
# get all tables -> carefull. This one is slow
getdict = dict(
    resultname=dict(
        whichfile="htm",
        as_tables=True,
    )
)

# get some rows in a html table
getdict = dict(
    resultname=dict(
        whichfile="htm",
        tableindex=1,  # or tablename
        # tablename="Site and Source Energy",  # tableindex takes priority if both given
        rows=[0, -1, 1],  # will return 3 rows as indexed
    )
)

# get some columns in a html table
getdict = dict(
    resultname=dict(
        whichfile="htm",
        tableindex=1,  # or tablename
        # tablename="Site to Source Energy Conversion Factors",  # tableindex takes priority if both given
        cols=[0, 1, -1],  # will return 3 columns as indexed
    )
)

# get some cells in a html table
getdict = dict(
    resultname=dict(
        whichfile="htm",
        tableindex=1,  # or tablename
        tablename="Site and Source Energy",  # tableindex takes priority if both given
        cells=[[0, -1], [1, -1], [-1, -1]],  # will return 3 cells
    ))

Now let us look at the csv files (this would be `resulttypes in ['csv', 'mcsv']`)

In [12]:
# get csv file cols
getdict = dict(
    resultname=dict(
        whichfile="csv",
        cols=[1, "Date/Time"], # you can give the index of the column or 
                               # the heading of the column
    )
)

That's it ! There are no functions to read and do a partial extract of the other file types.

Maybe the API should be changed so that you could send it a custom function and extract whatever you want. Worth a try in the future API

## I want to keep my result files - `runandget()`

What if I don't want to run my simulation anonymously. I want to keep the result files. Then you can use the `runandget()` function. Here is an example.

In [13]:
getdict = dict(
    HTML_file=dict(whichfile="htm", tableindex=0, table=True),
    end_file=dict(whichfile="end", entirefile=True),
)
runoptions = dict(
    output_suffix="D", output_prefix="Yousa", output_directory="./deletethislater", 
    readvars=True
)
fullresult = runandget.runandget(idf, runoptions, getdict)
pp.pprint(fullresult)


/Applications/EnergyPlus-9-3-0/energyplus --weather /Applications/EnergyPlus-9-3-0/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw --output-directory /Users/santoshphilip/Documents/coolshadow/github/witheppy/docs/deletethislater --idd /Applications/EnergyPlus-9-3-0/Energy+.idd --readvars --output-prefix Yousa --output-suffix D /Users/santoshphilip/Documents/coolshadow/github/witheppy/docs/in.idf

{'HTML_file': {'entirefile': None,
               'result': ['Site and Source Energy',
                          [['',
                            'Total Energy [GJ]',
                            'Energy Per Total Building Area [MJ/m2]',
                            'Energy Per Conditioned Building Area [MJ/m2]'],
                           ['Total Site Energy', 18.06, 77.76, 77.76],
                           ['Net Site Energy', 18.06, 77.76, 77.76],
                           ['Total Source Energy', 57.2, 246.26, 246.26],
                           ['Net Source Energy', 57.2, 246.26

The files are now in the `./deletethisfolder`. Let us check

In [14]:
import os
os.listdir(runoptions['output_directory'])

['Yousa.audit',
 'Yousa-sqlite.err',
 'Yousa.bnd',
 'Yousa.mtd',
 'Yousa.rvaudit',
 'Yousa.eso',
 'Yousa.csv',
 'Yousa.dxf',
 'Yousa.end',
 'Yousa-table.htm',
 'Yousa.eio',
 'Yousa.rdd',
 'Yousa.mtr',
 'Yousa.err',
 'Yousa-meter.csv',
 'Yousa.shd',
 'Yousa-table.tab',
 'Yousa.mdd']

Let us clean up and remove these files

In [15]:
for whichfile in runandget.resulttypes:
    fname = runandget.options2filename(whichfile, runoptions)
    try:
        os.remove(fname)
    except FileNotFoundError as e:
        pass
os.rmdir(runoptions['output_directory'])

## extract resutls without running the simulation - `getrun()`

What if the simulation takes a long time. I have already done the simulation. I know where the files are. I just want to extract the results. How do I do that ?

Lets just do a plain vanilla `idf.run()`

In [16]:
runoptions = dict(
    output_suffix="D", output_prefix="Yousa", output_directory="./deletethislater", 
    readvars=True
)

result = idf.run(**runoptions)
print(result)


/Applications/EnergyPlus-9-3-0/energyplus --weather /Applications/EnergyPlus-9-3-0/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw --output-directory /Users/santoshphilip/Documents/coolshadow/github/witheppy/docs/deletethislater --idd /Applications/EnergyPlus-9-3-0/Energy+.idd --readvars --output-prefix Yousa --output-suffix D /Users/santoshphilip/Documents/coolshadow/github/witheppy/docs/in.idf

None


I know the files are in the folder `./deletethislater`. I want to extract from the `*.end` file and the `*.htm` file

In [17]:
getdict = dict(
    HTML_file=dict(whichfile="htm", tableindex=0, table=True),
    end_file=dict(whichfile="end", entirefile=True),
)


fullresult = runandget.getrun(runoptions, getdict)
pp.pprint(fullresult)

{'HTML_file': {'entirefile': None,
               'result': ['Site and Source Energy',
                          [['',
                            'Total Energy [GJ]',
                            'Energy Per Total Building Area [MJ/m2]',
                            'Energy Per Conditioned Building Area [MJ/m2]'],
                           ['Total Site Energy', 18.06, 77.76, 77.76],
                           ['Net Site Energy', 18.06, 77.76, 77.76],
                           ['Total Source Energy', 57.2, 246.26, 246.26],
                           ['Net Source Energy', 57.2, 246.26, 246.26]]],
               'table': True,
               'tableindex': 0,
               'whichfile': 'htm'},
 'end_file': {'entirefile': True,
              'result': 'EnergyPlus Completed Successfully-- 4 Warning; 0 '
                        'Severe Errors; Elapsed Time=00hr 00min  3.70sec\n',
              'whichfile': 'end'}}


as easy as that. Now let us clean up the result directory again

In [18]:
for whichfile in runandget.resulttypes:
    fname = runandget.options2filename(whichfile, runoptions)
    try:
        os.remove(fname)
    except FileNotFoundError as e:
        pass
os.rmdir(runoptions['output_directory'])

## What else ?

Well ... We can return the files in `json` format. We can also compress the `json` files::

    runandget.anon_runandget(idf, getdict, json_it=True) 
    runandget.anon_runandget(idf, getdict, json_it=True, compress_it=True)     

## Limitations

- No error checking
    - for missing file
    - for missing table
    - for missing column or row
- Json format will work for binary files like `*.sql` or `*.htm`

It will just crash and burn :-)

## Motivation for this functionality

The primary motivation is to develop functions that can be used for distributed simulation. EnergyPlus generates a large volume of result files. You may be interested in only some data in these results, maybe even just one number - like the total energy use. In distributed simulation, some of your simulation nodes may be scattered over the internet and transfer of large files can be expensive and time consuming. So it would be useful to have a function in eppy that would run the simulation and return some specific results. the rest of the result files may be in a temporary location will be deleted.

Of cousrse to do a remote simulation, one has to send the `idf` to the node. Next step would be to make this function

